<a href="https://colab.research.google.com/github/Mporshe/Multimodal_Project/blob/main/multimodal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Copyright 2025 Google LLC and Makhanani Portia Ngoana
#
# Portions of this notebook include sample code provided by Google LLC,
# licensed under the Apache License, Version 2.0 (the "Apache License").
# You may not use those portions except in compliance with the Apache License.
# You may obtain a copy of the Apache License at:
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# My original contributions in this notebook are licensed under the
# Creative Commons Attribution 4.0 International License (CC BY 4.0),
# in accordance with the Kaggle BigQuery AI Hackathon rules:
#
#     https://creativecommons.org/licenses/by/4.0/
#
# Unless required by applicable law or agreed to in writing, the software
# distributed under either license is provided on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the respective license texts for the specific language governing
# permissions and limitations under each license.


## Set up your Google Cloud Project  

Before running this notebook, please ensure the following steps are completed:  

1. **Select or create a Google Cloud project**  
   - [Google Cloud Console](https://console.cloud.google.com/)  
   - New accounts receive **$300 free credit** for compute/storage costs.  

2. **Enable billing** for your project.  

3. **Enable required APIs**:  
   - BigQuery API  
   - BigQuery Connection API  
   - Vertex AI API  

4. **(Optional, if running locally)** Install the [Cloud SDK](https://cloud.google.com/sdk) to run `gcloud` commands.  

---


**Set your project ID**

In [ ]:

PROJECT_ID = "ecstatic-gantry-470611-m5"

# Set the project id
! gcloud config set project {PROJECT_ID}

Updated property [core/project].


**Authenticate to your Google Cloud account**

In [ ]:

from google.colab import auth
auth.authenticate_user()


**Local JupyterLab instance,uncomment and run the following:**

In [ ]:

#! gcloud auth login


**Create BigQuery Cloud resource connection**

In [ ]:
!bq mk --connection --location=us \
    --connection_type=CLOUD_RESOURCE test_connection

BigQuery error in mk operation: Already Exists: Connection
projects/392599610586/locations/us/connections/test_connection


**Set permissions for Service Account**

In [ ]:
SERVICE_ACCT = !bq show --format=prettyjson --connection us.test_connection | grep "serviceAccountId" | cut -d '"' -f 4
SERVICE_ACCT_EMAIL = SERVICE_ACCT[-1]
print(SERVICE_ACCT_EMAIL)

bqcx-392599610586-ooa4@gcp-sa-bigquery-condel.iam.gserviceaccount.com


In [ ]:
import time

PROJECT_ID = "ecstatic-gantry-470611-m5"
SERVICE_ACCT_EMAIL = "bqcx-392599610586-ooa4@gcp-sa-bigquery-condel.iam.gserviceaccount.com"

# Give Storage Object Viewer
!gcloud projects add-iam-policy-binding $PROJECT_ID \
  --member="serviceAccount:$SERVICE_ACCT_EMAIL" \
  --role="roles/storage.objectViewer"

# Give Vertex AI User
!gcloud projects add-iam-policy-binding $PROJECT_ID \
  --member="serviceAccount:$SERVICE_ACCT_EMAIL" \
  --role="roles/aiplatform.user"

# Wait ~60s for IAM changes to propagate
time.sleep(60)

Updated IAM policy for project [ecstatic-gantry-470611-m5].
bindings:
- members:
  - serviceAccount:service-392599610586@gcp-sa-vertex-nb.iam.gserviceaccount.com
  role: roles/aiplatform.colabServiceAgent
- members:
  - serviceAccount:service-392599610586@gcp-sa-aiplatform-vm.iam.gserviceaccount.com
  role: roles/aiplatform.notebookServiceAgent
- members:
  - serviceAccount:service-392599610586@gcp-sa-aiplatform.iam.gserviceaccount.com
  role: roles/aiplatform.serviceAgent
- members:
  - serviceAccount:bqcx-392599610586-gqt9@gcp-sa-bigquery-condel.iam.gserviceaccount.com
  - serviceAccount:bqcx-392599610586-o7yp@gcp-sa-bigquery-condel.iam.gserviceaccount.com
  - serviceAccount:bqcx-392599610586-ooa4@gcp-sa-bigquery-condel.iam.gserviceaccount.com
  role: roles/aiplatform.user
- members:
  - serviceAccount:392599610586@cloudbuild.gserviceaccount.com
  role: roles/cloudbuild.builds.builder
- members:
  - serviceAccount:service-392599610586@gcp-sa-cloudbuild.iam.gserviceaccount.com
  role:

**Create a BigQuery Dataset**


In [ ]:
%%bigquery --project ecstatic-gantry-470611-m5
CREATE SCHEMA IF NOT EXISTS `ecstatic-gantry-470611-m5.cars`
OPTIONS(location='US');



Query is running:   0%|          |

""


**Set Colab display options**

In [ ]:
%load_ext google.colab.data_table

**Created a Cloud Storage bucket called "mycars-dataset-2025" under my project_ID**

In [ ]:

PROJECT_ID="ecstatic-gantry-470611-m5"
BUCKET_NAME="ecstatic-gantry-470611-m5-mycars-dataset-2025"
REGION="US"


## Data Sources  
1. **Structured Data (Vehicle Specifications)**  
   - URL: https://www.kaggle.com/datasets/abdulmalik1518/cars-datasets-2025
   - Contains: Car company, model, engine type, horsepower, top speed, acceleration (0–100 km/h), price, seating capacity, fuel type, torque, and battery capacity.  


2. **Unstructured Data (Vehicle Images)**  
   - Source: KaggleHub (dataset: *The Car Connection Picture Dataset*)  
   - Code to acess the data:  
     ```python
     import kagglehub
     path = kagglehub.dataset_download("prondeau/the-car-connection-picture-dataset")
     print("Path to dataset files:", path)
     ```  
   - Contains: Car images  

**Stored both dataset into my Bucket under my project_ID**

In [ ]:

!gsutil ls gs://$BUCKET_NAME

gs://ecstatic-gantry-470611-m5-mycars-dataset-2025/cars.csv
gs://ecstatic-gantry-470611-m5-mycars-dataset-2025/cars_images/


In [ ]:
!gcloud storage buckets create gs://$BUCKET_NAME \
  --project=$PROJECT_ID --location=$REGION --uniform-bucket-level-access

Creating gs://ecstatic-gantry-470611-m5-mycars-dataset-2025/...
ERROR: (gcloud.storage.buckets.create) HTTPError 409: Your previous request to create the named bucket succeeded and you already own it.


**Load Structured Data (SQL)**

In [ ]:
%%bigquery --project $PROJECT_ID
LOAD DATA OVERWRITE `ecstatic-gantry-470611-m5.cars.carsales_data`
(
  Car_Company_Names   STRING,
  Car_Models          STRING,
  Engine_Types        STRING,
  CC_Battery_Capacity STRING,
  Horsepower_HP       STRING,
  Top_Speed           STRING,
  Accel_0_100_kmh     STRING,
  Price_USD           STRING,
  Fuel_Type           STRING,
  Seating_Capacity    STRING,
  Torque              STRING
)
FROM FILES (
  uris = ['gs://ecstatic-gantry-470611-m5-mycars-dataset-2025/cars.csv'],
  format = 'CSV',
  skip_leading_rows = 1
);




Query is running:   0%|          |

""


**Preview Structured Cars Dataset**

In [ ]:
%%bigquery --project {PROJECT_ID}
SELECT *
FROM `ecstatic-gantry-470611-m5.cars.carsales_data`


Query is running:   0%|          |

Downloading:   0%|          |

,Car_Company_Names,Car_Models,Engine_Types,CC_Battery_Capacity,Horsepower_HP,Top_Speed,Accel_0_100_kmh,Price_USD,Fuel_Type,Seating_Capacity,Torque
0,MARUTI SUZUKI,SWIFT,"1.2L,4-CYLINDER,INLINE-4(I4)","1,197 cc",83 hp,160 km/h,12.2 sec,"$10,400",Petrol,5,113 Nm
1,MARUTI SUZUKI,DZIRE,"1.2L,4-CYLINDER,INLINE-4(I4)","1,197 cc",83 hp,160 km/h,12.2 sec,"$11,400",Petrol/Diesel,5,113 Nm
2,MARUTI SUZUKI,BREZZA,"1.5L,4-CYLINDER,INLINE(I4)","1,462 cc",103 hp,180 km/h,11.5 sec,"$14,400",Petrol,5,200 Nm
3,Toyota,C-HR+,1.8L / 2.0L Hybrid,1798 / 1987 cc + batt,140 – 198 hp,180 km/h,7.9 – 10.5 sec,"€ 33,000",Hybrid,5,190 – 205 Nm
4,Toyota,Corolla Cross,2.0L Gas / 2.0L Hybrid,1987 cc / Hybrid batt,169 – 196 hp,190 km/h,8.0 – 9.2 sec,"$25,210 – $29,135",Gas / Hybrid,5,190 – 210 Nm
...,...,...,...,...,...,...,...,...,...,...,...
1213,Volvo,Volvo FH,13.0L I6 Turbo Diesel,13000 cc,540 hp,120 km/h,15.0 sec,"$120,000",Diesel,2,2600 Nm
1214,Volvo,Volvo FMX,13.0L I6 Turbo Diesel,13000 cc,500 hp,110 km/h,16.5 sec,"$110,000",Diesel,2,2500 Nm
1215,Volvo,Volvo FH16,16.1L I6 Turbo Diesel,16100 cc,750 hp,120 km/h,14.5 sec,"$160,000",Diesel,2,3500 Nm
1216,Volvo,Volvo FL,5.1L I4 Turbo Diesel,5100 cc,180 hp,80 km/h,22.0 sec,"$75,000",Diesel,2,800 Nm


**Load Unstructured Data (Object Table for Car Images)**

In [ ]:
%%bigquery --project ecstatic-gantry-470611-m5
CREATE OR REPLACE EXTERNAL TABLE `ecstatic-gantry-470611-m5.cars.car_images`
WITH CONNECTION `us.test_connection`
OPTIONS (
  object_metadata = 'SIMPLE',
  uris = ['gs://ecstatic-gantry-470611-m5-mycars-dataset-2025/cars_images/*']
);


Query is running:   0%|          |

""


**Preview Object Table for Car Images**

In [ ]:
%%bigquery --project ecstatic-gantry-470611-m5
SELECT*
FROM `ecstatic-gantry-470611-m5.cars.car_images`


Query is running:   0%|          |

Downloading:   0%|          |

,uri,generation,content_type,size,md5_hash,updated,metadata,ref
0,gs://ecstatic-gantry-470611-m5-mycars-dataset-...,1756996651616043,image/jpeg,16064,67b52d6dbce56cddc6f19cdd2f659276,2025-09-04 14:37:31.637000+00:00,[],{'uri': 'gs://ecstatic-gantry-470611-m5-mycars...
1,gs://ecstatic-gantry-470611-m5-mycars-dataset-...,1756997276746751,image/jpeg,7587,a390be726f9a5b4e2a09e31a9deec76f,2025-09-04 14:47:56.781000+00:00,[],{'uri': 'gs://ecstatic-gantry-470611-m5-mycars...
2,gs://ecstatic-gantry-470611-m5-mycars-dataset-...,1756997732668831,image/jpeg,11840,ecbdd93130c26329ffab485dd78e5a35,2025-09-04 14:55:32.691000+00:00,[],{'uri': 'gs://ecstatic-gantry-470611-m5-mycars...
3,gs://ecstatic-gantry-470611-m5-mycars-dataset-...,1757000338211692,image/jpeg,8417,ca8232a7bcc4085ad667f455a8e4cc81,2025-09-04 15:38:58.246000+00:00,[],{'uri': 'gs://ecstatic-gantry-470611-m5-mycars...
4,gs://ecstatic-gantry-470611-m5-mycars-dataset-...,1756997114096789,image/jpeg,6908,5458cf6d46aaed97921605b647f034a0,2025-09-04 14:45:14.133000+00:00,[],{'uri': 'gs://ecstatic-gantry-470611-m5-mycars...
...,...,...,...,...,...,...,...,...
64783,gs://ecstatic-gantry-470611-m5-mycars-dataset-...,1757004731942618,image/jpeg,10247,53ab917ee1b91c53ea8be331fbdd5d77,2025-09-04 16:52:11.983000+00:00,[],{'uri': 'gs://ecstatic-gantry-470611-m5-mycars...
64784,gs://ecstatic-gantry-470611-m5-mycars-dataset-...,1757004739752567,image/jpeg,13155,31bec78472dcc8bfb0b471fd60545112,2025-09-04 16:52:19.774000+00:00,[],{'uri': 'gs://ecstatic-gantry-470611-m5-mycars...
64785,gs://ecstatic-gantry-470611-m5-mycars-dataset-...,1757004736222821,image/jpeg,9532,2b7b4ecf31b6a192940907d90ae5ba3c,2025-09-04 16:52:16.245000+00:00,[],{'uri': 'gs://ecstatic-gantry-470611-m5-mycars...
64786,gs://ecstatic-gantry-470611-m5-mycars-dataset-...,1757004721822459,image/jpeg,9588,17d79489e237ee0bcf8452ede5dea561,2025-09-04 16:52:01.845000+00:00,[],{'uri': 'gs://ecstatic-gantry-470611-m5-mycars...


**Builds a structured lookup table (make_key/model_key + image metadata) out of your raw image filenames in Cloud Storage**

In [ ]:
%%bigquery --project ecstatic-gantry-470611-m5
CREATE OR REPLACE TABLE `ecstatic-gantry-470611-m5.cars.img_keys`
PARTITION BY DATE(updated)
CLUSTER BY make_key, model_key AS
WITH base AS (
  SELECT
    uri,
    REGEXP_EXTRACT(uri, r'/([^/]+)$') AS name,
    content_type,
    size,
    updated,
    ref AS object_ref
  FROM `ecstatic-gantry-470611-m5.cars.car_images`
)
SELECT
  name,
  uri,
  content_type,
  size,
  updated,

  -- Make = everything before the first underscore
  TRIM(REGEXP_EXTRACT(name, r'^([^_]+)'))            AS make_raw,

  -- Model = text between first and second underscores
  TRIM(REGEXP_EXTRACT(name, r'^[^_]+_([^_]+)'))      AS model_raw,

  -- Normalize for joins (UPPER + strip non-alphanumerics)
  UPPER(REGEXP_REPLACE(TRIM(REGEXP_EXTRACT(name, r'^([^_]+)')),       r'[^A-Za-z0-9]', '')) AS make_key,
  UPPER(REGEXP_REPLACE(TRIM(REGEXP_EXTRACT(name, r'^[^_]+_([^_]+)')), r'[^A-Za-z0-9]', '')) AS model_key,

  object_ref
FROM base;


Query is running:   0%|          |

""


**Counts total image records**

In [ ]:
%%bigquery --project ecstatic-gantry-470611-m5
SELECT COUNT(*) AS n_images FROM `ecstatic-gantry-470611-m5.cars.img_keys`;

Query is running:   0%|          |

Downloading:   0%|          |

,n_images
0,64788


**Normalizes the structured car data (company + model → make_key / model_key) and clusters it.**

In [ ]:
%%bigquery --project ecstatic-gantry-470611-m5
CREATE OR REPLACE TABLE `ecstatic-gantry-470611-m5.cars.cars_keys`
CLUSTER BY make_key, model_key AS
SELECT
  Car_Company_Names,
  Car_Models,
  Engine_Types,
  CC_Battery_Capacity,
  Horsepower_HP,
  Top_Speed,
  Accel_0_100_kmh,
  Price_USD,
  Fuel_Type,
  Seating_Capacity,
  Torque,

  -- Robust keys: UPPER first, strip non-alnum, trim, null-safe
  REGEXP_REPLACE( UPPER(COALESCE(TRIM(Car_Company_Names), '')), r'[^A-Z0-9]', '' ) AS make_key,
  REGEXP_REPLACE( UPPER(COALESCE(TRIM(Car_Models),        '')), r'[^A-Z0-9]', '' ) AS model_key

FROM `ecstatic-gantry-470611-m5.cars.carsales_data`;

Query is running:   0%|          |

""


**Counts how many rows are in the Structured car data normalized table**

In [ ]:
%%bigquery --project ecstatic-gantry-470611-m5
SELECT COUNT(*) AS n_rows FROM `ecstatic-gantry-470611-m5.cars.cars_keys`;

Query is running:   0%|          |

Downloading:   0%|          |

,n_rows
0,1218


**Create a multimodal table that joins the cleaned structured car data (cars_keys) with the cleaned image metadata (img_keys) on make_key and model_key**

In [ ]:
%%bigquery --project ecstatic-gantry-470611-m5
CREATE OR REPLACE TABLE `ecstatic-gantry-470611-m5.cars.cars_multimodal` AS
WITH joined AS (
  SELECT
    c.*,
    i.uri,
    i.name,
    i.object_ref AS ref,
    i.updated,
    i.size,
    i.content_type,

    -- Normalize image filename so we can do substring checks safely
    REGEXP_REPLACE(UPPER(i.name), r'[^A-Z0-9]', '') AS name_key,

    -- Score: exact key match (+10 each), filename contains model token (+3),
    -- MIME is image/* (+1). Keys match by USING(), but keep weights explicit.
    (CASE WHEN i.make_key = c.make_key  THEN 10 ELSE 0 END) +
    (CASE WHEN i.model_key = c.model_key THEN 10 ELSE 0 END) +
    (CASE WHEN INSTR(REGEXP_REPLACE(UPPER(i.name), r'[^A-Z0-9]', ''), c.model_key) > 0 THEN 3 ELSE 0 END) +
    (CASE WHEN STARTS_WITH(i.content_type, 'image/') THEN 1 ELSE 0 END)
    AS score
  FROM `ecstatic-gantry-470611-m5.cars.cars_keys` AS c
  INNER JOIN `ecstatic-gantry-470611-m5.cars.img_keys`  AS i
  USING (make_key, model_key)
),
ranked AS (
  SELECT
    *,
    ROW_NUMBER() OVER (
      PARTITION BY make_key, model_key
      ORDER BY score DESC, updated DESC, size DESC, name
    ) AS rn
  FROM joined
)
SELECT * EXCEPT(name_key, score, rn, updated, size, content_type,make_key,model_key,name)
FROM ranked
WHERE rn = 1;


Query is running:   0%|          |

""


**Preview of the combined structued and unstructured cars data called the multimodal table**

In [ ]:
%%bigquery --project {PROJECT_ID}
SELECT *
FROM `ecstatic-gantry-470611-m5.cars.cars_multimodal`

Query is running:   0%|          |

Downloading:   0%|          |

,Car_Company_Names,Car_Models,Engine_Types,CC_Battery_Capacity,Horsepower_HP,Top_Speed,Accel_0_100_kmh,Price_USD,Fuel_Type,Seating_Capacity,Torque,uri,ref
0,ASTON MARTIN,DB11,V12,"5,204 cc",600 hp,322 km/h,3.9 sec,"$208,000",Petrol,4,700 Nm,gs://ecstatic-gantry-470611-m5-mycars-dataset-...,{'uri': 'gs://ecstatic-gantry-470611-m5-mycars...
1,ASTON MARTIN,VANQUISH,V12,"5,935 cc",580 hp,324 km/h,3.6 sec,"$294,000",Petrol,2,630 Nm,gs://ecstatic-gantry-470611-m5-mycars-dataset-...,{'uri': 'gs://ecstatic-gantry-470611-m5-mycars...
2,ASTON MARTIN,VANTAGE,V8,"3,996 cc",503 hp,313 km/h,3.6 sec,"$142,000",Petrol,2,685 Nm,gs://ecstatic-gantry-470611-m5-mycars-dataset-...,{'uri': 'gs://ecstatic-gantry-470611-m5-mycars...
3,AUDI,E-TRON,ELECTRIC,95 kwh,402 hp,200 km/h,5.5 sec,"$75,000",Electric,5,664 Nm,gs://ecstatic-gantry-470611-m5-mycars-dataset-...,{'uri': 'gs://ecstatic-gantry-470611-m5-mycars...
4,AUDI,A4,I4,"1,984 cc",248 hp,240 km/h,6.5 sec,"$39,000",Petrol,5,400 Nm,gs://ecstatic-gantry-470611-m5-mycars-dataset-...,{'uri': 'gs://ecstatic-gantry-470611-m5-mycars...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
130,Volkswagen,Golf,1.5L Turbo I4 / Electric,1498 cc / 40 - 58 kWh,110 - 320 hp,210 km/h,8.5 sec,"$23,000",Petrol,5,250 Nm,gs://ecstatic-gantry-470611-m5-mycars-dataset-...,{'uri': 'gs://ecstatic-gantry-470611-m5-mycars...
131,Volkswagen,Passat,2.0L Turbo I4,1984 cc,150 - 280 hp,210 km/h,7.8 sec,"$25,000",Petrol,5,250 Nm,gs://ecstatic-gantry-470611-m5-mycars-dataset-...,{'uri': 'gs://ecstatic-gantry-470611-m5-mycars...
132,Volkswagen,Arteon,2.0L Turbo I4,1984 cc,150 - 280 hp,250 km/h,5.6 sec,"$46,000",Petrol,5,350 Nm,gs://ecstatic-gantry-470611-m5-mycars-dataset-...,{'uri': 'gs://ecstatic-gantry-470611-m5-mycars...
133,Volkswagen,Tiguan,2.0L Turbo I4,1984 cc,150 - 230 hp,209 km/h,8.5 sec,"$28,505","Petrol, Hybrid",7,300 Nm,gs://ecstatic-gantry-470611-m5-mycars-dataset-...,{'uri': 'gs://ecstatic-gantry-470611-m5-mycars...


**From the multimodal table (cars_multimodal), send each car image reference (ref) into Gemini with AI.GENERATE_TABLE to create a new enriched table that adds AI-generated fields — a one-sentence image description, color, body style, and number of doors.**

In [ ]:
%%bigquery --project ecstatic-gantry-470611-m5
CREATE OR REPLACE TABLE `ecstatic-gantry-470611-m5.cars.cars_multimodal_enriched` AS
WITH gen AS (
  SELECT *
  FROM AI.GENERATE_TABLE(
    MODEL `ecstatic-gantry-470611-m5.DEV.gemini`,
    (
      SELECT
        (
          CONCAT(
            'Describe this car in one sentence. Extract: ',
            'color, body_style, num_doors. ',
            'Return exactly: ',
            'ai_image_description STRING, ai_color STRING, ai_body_style STRING, ai_num_doors INT64. '
          ),
          OBJ.GET_ACCESS_URL(ref, 'r')
        ) AS prompt,
        Car_Company_Names, Car_Models, Engine_Types, CC_Battery_Capacity,
        Horsepower_HP, Top_Speed, Accel_0_100_kmh, Price_USD, Fuel_Type,
        Seating_Capacity, Torque, uri, ref
      FROM `ecstatic-gantry-470611-m5.cars.cars_multimodal`
      WHERE ref IS NOT NULL
    ),
    STRUCT(
      "ai_image_description STRING, ai_color STRING, ai_body_style STRING, ai_num_doors INT64" AS output_schema,
      512 AS max_output_tokens,
      0.2 AS temperature
    )
  )
)
SELECT
  Car_Company_Names, Car_Models, Engine_Types, CC_Battery_Capacity,
  Horsepower_HP, Top_Speed, Accel_0_100_kmh, Price_USD, Fuel_Type,
  Seating_Capacity, Torque, ai_image_description, uri, ref

FROM gen;



Query is running:   0%|          |

""


**Preview: Enriched table (cars_multimodal_enriched), which combines the structured car data with AI-extracted visual features from the images.**

In [ ]:
%%bigquery --project {PROJECT_ID}
SELECT *
FROM `ecstatic-gantry-470611-m5.cars.cars_multimodal_enriched`

Query is running:   0%|          |

Downloading:   0%|          |

,Car_Company_Names,Car_Models,Engine_Types,CC_Battery_Capacity,Horsepower_HP,Top_Speed,Accel_0_100_kmh,Price_USD,Fuel_Type,Seating_Capacity,Torque,ai_image_description,uri,ref
0,ASTON MARTIN,DB11,V12,"5,204 cc",600 hp,322 km/h,3.9 sec,"$208,000",Petrol,4,700 Nm,A bronze Aston Martin convertible is parked in...,gs://ecstatic-gantry-470611-m5-mycars-dataset-...,{'uri': 'gs://ecstatic-gantry-470611-m5-mycars...
1,ASTON MARTIN,VANQUISH,V12,"5,935 cc",580 hp,324 km/h,3.6 sec,"$294,000",Petrol,2,630 Nm,This is a white Aston Martin Vanquish coupe wi...,gs://ecstatic-gantry-470611-m5-mycars-dataset-...,{'uri': 'gs://ecstatic-gantry-470611-m5-mycars...
2,ASTON MARTIN,VANTAGE,V8,"3,996 cc",503 hp,313 km/h,3.6 sec,"$142,000",Petrol,2,685 Nm,"This is a silver car with the hood open, showc...",gs://ecstatic-gantry-470611-m5-mycars-dataset-...,{'uri': 'gs://ecstatic-gantry-470611-m5-mycars...
3,AUDI,E-TRON,ELECTRIC,95 kwh,402 hp,200 km/h,5.5 sec,"$75,000",Electric,5,664 Nm,This is a silver Audi e-tron SUV with four doors.,gs://ecstatic-gantry-470611-m5-mycars-dataset-...,{'uri': 'gs://ecstatic-gantry-470611-m5-mycars...
4,AUDI,A4,I4,"1,984 cc",248 hp,240 km/h,6.5 sec,"$39,000",Petrol,5,400 Nm,A silver Audi sedan is parked on a paved surfa...,gs://ecstatic-gantry-470611-m5-mycars-dataset-...,{'uri': 'gs://ecstatic-gantry-470611-m5-mycars...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
130,Volkswagen,Golf,1.5L Turbo I4 / Electric,1498 cc / 40 - 58 kWh,110 - 320 hp,210 km/h,8.5 sec,"$23,000",Petrol,5,250 Nm,This is a front view of a white Volkswagen Gol...,gs://ecstatic-gantry-470611-m5-mycars-dataset-...,{'uri': 'gs://ecstatic-gantry-470611-m5-mycars...
131,Volkswagen,Arteon,2.0L Turbo I4,1984 cc,150 - 280 hp,250 km/h,5.6 sec,"$46,000",Petrol,5,350 Nm,The interior of a Volkswagen Passat shows a bl...,gs://ecstatic-gantry-470611-m5-mycars-dataset-...,{'uri': 'gs://ecstatic-gantry-470611-m5-mycars...
132,Volkswagen,Passat,2.0L Turbo I4,1984 cc,150 - 280 hp,210 km/h,7.8 sec,"$25,000",Petrol,5,250 Nm,This is a gray Volkswagen Passat sedan.,gs://ecstatic-gantry-470611-m5-mycars-dataset-...,{'uri': 'gs://ecstatic-gantry-470611-m5-mycars...
133,Volkswagen,Beetle,2.0L Turbo I4,1984 cc,105 - 210 hp,205 km/h,7.0 sec,"$20,000",Petrol,4,240 Nm,A silver Volkswagen Beetle with a rounded body...,gs://ecstatic-gantry-470611-m5-mycars-dataset-...,{'uri': 'gs://ecstatic-gantry-470611-m5-mycars...


**Registers Gemini model remotely.**

In [ ]:
%%bigquery --project ecstatic-gantry-470611-m5
CREATE OR REPLACE MODEL `cars.gemini`
REMOTE WITH CONNECTION `us.test_connection`
OPTIONS (endpoint = 'gemini-2.5-flash');


Query is running:   0%|          |

""


**Permissions on your Cloud Storage Bucket**

In [ ]:
!gcloud storage buckets add-iam-policy-binding gs://ecstatic-gantry-470611-m5-mycars-dataset-2025 \
  --member=allUsers --role=roles/storage.objectViewer

bindings:
- members:
  - projectEditor:ecstatic-gantry-470611-m5
  - projectOwner:ecstatic-gantry-470611-m5
  role: roles/storage.legacyBucketOwner
- members:
  - projectViewer:ecstatic-gantry-470611-m5
  role: roles/storage.legacyBucketReader
- members:
  - projectEditor:ecstatic-gantry-470611-m5
  - projectOwner:ecstatic-gantry-470611-m5
  role: roles/storage.legacyObjectOwner
- members:
  - projectViewer:ecstatic-gantry-470611-m5
  role: roles/storage.legacyObjectReader
- members:
  - allUsers
  role: roles/storage.objectViewer
etag: CAk=
kind: storage#policy
resourceId: projects/_/buckets/ecstatic-gantry-470611-m5-mycars-dataset-2025
version: 1


**Run a multimodal query**

**1) Most expensive true sports cars (highest to lowest)**

In [ ]:
%%bigquery df --project ecstatic-gantry-470611-m5
SELECT
  Car_Company_Names  AS company_name,
  Car_Models         AS model_name,
  Engine_Types,
  Price_USD,
  ai_image_description,
  REGEXP_REPLACE(uri, r'^gs://([^/]+)/(.+)$',
                 'https://storage.googleapis.com/\\1/\\2') AS image_url,
  SAFE_CAST(REGEXP_REPLACE(Price_USD, r'[^0-9.]', '') AS FLOAT64) AS price_usd_numeric
FROM `ecstatic-gantry-470611-m5.cars.cars_multimodal_enriched`
WHERE
  SAFE_CAST(REGEXP_REPLACE(Price_USD, r'[^0-9.]', '') AS FLOAT64) > 280000
  AND AI.GENERATE_BOOL(
        prompt => (
          'Does this image clearly show a high-performance sports car (2-door coupe/convertible)? Return TRUE only if clearly yes.',
          ref
        ),
        connection_id => 'us.test_connection'
      ).result
ORDER BY price_usd_numeric ASC;





Query is running:   0%|          |

Downloading:   0%|          |

**Preview: Most expensive true sports cars (highest to lowest)**

In [ ]:
from IPython.display import HTML, display
import pandas as pd

df_show = df.copy()
df_show["thumb"] = df_show["image_url"].fillna("").apply(
    lambda u: f"<img src='{u}' width='100'>" if u else ""
)

cols = ["thumb","company_name","model_name","Engine_Types","Price_USD","ai_image_description"]
display(HTML(df_show[cols].to_html(escape=False, index=False)))



thumb,company_name,model_name,Engine_Types,Price_USD,ai_image_description
,ASTON MARTIN,VANQUISH,V12,"$294,000",This is a white Aston Martin Vanquish coupe with a sleek design.
,ROLLS ROYCE,WRAITH,V12,"$330,000",This is a blue Rolls-Royce Wraith coupe with two doors.
,ROLLS ROYCE,DAWN,V12,"$360,000",This is a dark gray Rolls-Royce Dawn convertible with orange interior and suicide doors parked on a paved surface with a grassy area and trees in the background.
,Mazda,MX-5 Miata,2.0L SkyActiv-G,"$27,000 - $33,000","This is a white Mazda Miata convertible with two doors, seen from the rear."


**2) Top 2 Fast cars by specs (accel < 4s AND top speed > 300 km/h)**

In [ ]:
%%bigquery df --project ecstatic-gantry-470611-m5
SELECT
  Car_Company_Names,
  Car_Models,
  Top_Speed,
  Accel_0_100_kmh,
  Price_USD,
  ai_image_description,
  REGEXP_REPLACE(uri, r'^gs://([^/]+)/(.+)$',
                 'https://storage.googleapis.com/\\1/\\2') AS image_url,
  SAFE_CAST(REGEXP_REPLACE(Price_USD, r'[^0-9.]', '') AS FLOAT64) AS price_usd_numeric
FROM `ecstatic-gantry-470611-m5.cars.cars_multimodal_enriched`
WHERE SAFE_CAST(REGEXP_EXTRACT(Top_Speed, r'([0-9.]+)') AS FLOAT64) > 300
  AND SAFE_CAST(REGEXP_EXTRACT(Accel_0_100_kmh, r'([0-9.]+)') AS FLOAT64) < 4
ORDER BY price_usd_numeric DESC
LIMIT 2


Query is running:   0%|          |

Downloading:   0%|          |

**Preview:Top 2 Fast cars by specs (accel < 4s AND top speed > 300 km/h)**

In [ ]:
from IPython.display import HTML, display
import pandas as pd

df_show = df.copy()

# thumbnail
df_show["thumb"] = df_show["image_url"].fillna("").apply(
    lambda u: f"<img src='{u}' width='150'>" if u else ""
)

# optional: shorten long descriptions so the table stays compact
def shorten(s, n=200):
    if not isinstance(s, str):
        return ""
    return s if len(s) <= n else s[:n-1] + "…"

df_show["ai_image_description_short"] = df_show["ai_image_description"].apply(shorten)

cols = [
    "thumb",
    "Car_Company_Names", "Car_Models",
    "Top_Speed", "Accel_0_100_kmh", "Price_USD",
    "ai_image_description_short"
]

display(HTML(df_show[cols].to_html(escape=False, index=False)))





thumb,Car_Company_Names,Car_Models,Top_Speed,Accel_0_100_kmh,Price_USD,ai_image_description_short
,ASTON MARTIN,VANQUISH,324 km/h,3.6 sec,"$294,000",This is a white Aston Martin Vanquish coupe with a sleek design.
,LAMBORGHINI,URUS,305 km/h,3.6 sec,"$211,000",This is a yellow Lamborghini Urus SUV.


**3) Cars with hood open (QA/curation)**

In [ ]:
%%bigquery df --project ecstatic-gantry-470611-m5
SELECT
  Car_Company_Names,
  Car_Models,
  ai_image_description,
  -- turn gs://... into a public HTTPS URL
  REGEXP_REPLACE(uri, r'^gs://([^/]+)/(.+)$',
                 'https://storage.googleapis.com/\\1/\\2') AS image_url
FROM `ecstatic-gantry-470611-m5.cars.cars_multimodal_enriched`
WHERE AI.GENERATE_BOOL(
        prompt => ('Does the image show a car with the hood open?', ref),
        connection_id => 'us.test_connection'
      ).result;


Query is running:   0%|          |

Downloading:   0%|          |

**Preview: Cars with hood open (QA/curation)**

In [ ]:
from IPython.display import HTML, display

df_show = df.copy()
df_show["thumb"] = df_show["image_url"].fillna("").apply(
    lambda u: f"<img src='{u}' width='150'>" if u else ""
)

cols = ["thumb", "Car_Company_Names", "Car_Models", "ai_image_description"]
display(HTML(df_show[cols].to_html(escape=False, index=False)))


thumb,Car_Company_Names,Car_Models,ai_image_description
,ASTON MARTIN,VANTAGE,"This is a silver car with the hood open, showcasing the engine."
,Mitsubishi,Mirage,"A pink Mitsubishi Mirage with its hood open, revealing the engine."
,NISSAN,SENTRA,A group of people are gathered around a silver four-door car outdoors.


**4) Top 4 number of high-performance sports cars per brand (AI + price)**

In [ ]:
%%bigquery df --project ecstatic-gantry-470611-m5
SELECT
  Car_Company_Names AS company_name,
  COUNT(*) AS n_sporty_count,
  ANY_VALUE(REGEXP_REPLACE(uri, r'^gs://([^/]+)/(.+)$',
                           'https://storage.googleapis.com/\\1/\\2')) AS image_url,
  ANY_VALUE(ai_image_description) AS ai_image_description
FROM `ecstatic-gantry-470611-m5.cars.cars_multimodal_enriched`
WHERE SAFE_CAST(REGEXP_REPLACE(Price_USD, r'[^0-9.]','') AS FLOAT64) > 100000
  AND AI.GENERATE_BOOL(
        prompt => ('Is this a 2-door sports car (coupe/convertible)? Return TRUE only if clearly yes.', ref),
        connection_id => 'us.test_connection'
      ).result
GROUP BY company_name
ORDER BY n_sporty_count DESC
LIMIT 4



Query is running:   0%|          |

Downloading:   0%|          |

**Preview:Top 4 number of high-performance sports cars per brand (AI + price)**

In [ ]:
from IPython.display import HTML, display

df_show = df.copy()

# create thumbnail column
df_show["thumb"] = df_show["image_url"].fillna("").apply(
    lambda u: f"<img src='{u}' width='120'>" if u else ""
)

# display with images + counts + descriptions
cols = ["thumb", "company_name", "n_sporty_count", "ai_image_description"]
display(HTML(df_show[cols].to_html(escape=False, index=False)))




thumb,company_name,n_sporty_count,ai_image_description
,ROLLS ROYCE,3,This is a dark gray Rolls-Royce Dawn convertible with orange interior and suicide doors parked on a paved surface with a grassy area and trees in the background.
,ASTON MARTIN,2,This is a white Aston Martin Vanquish coupe with a sleek design.
,NISSAN,1,This is a vibrant orange Nissan GT-R coupe with a sleek body and two doors.
,FERRARI,1,"A vibrant red Mazda MX-5 Miata RF cruises down a sunlit road, showcasing its sleek coupe design."


**5) Top 6 "red cars” only (image-based color filter)  including price and ai image description**

In [ ]:
%%bigquery df --project ecstatic-gantry-470611-m5
SELECT
  Car_Company_Names,
  Car_Models,
  Price_USD,
  ai_image_description,
  REGEXP_REPLACE(uri, r'^gs://([^/]+)/(.+)$',
                 'https://storage.googleapis.com/\\1/\\2') AS image_url
FROM `ecstatic-gantry-470611-m5.cars.cars_multimodal_enriched`
WHERE AI.GENERATE_BOOL(
        prompt => ('Is the primary body color predominantly red?', ref),
        connection_id => 'us.test_connection'
      ).result
  AND SAFE_CAST(REGEXP_REPLACE(Price_USD, r'[^0-9.]','') AS FLOAT64) IS NOT NULL
ORDER BY SAFE_CAST(REGEXP_REPLACE(Price_USD, r'[^0-9.]','') AS FLOAT64) DESC LIMIT 6;


Query is running:   0%|          |

Downloading:   0%|          |

**Preview: Top 6 "red cars” only (image-based color filter) including price and ai image description**

In [ ]:
from IPython.display import HTML, display

df_show = df.copy()

# turn image_url into an <img> tag
df_show["thumb"] = df_show["image_url"].fillna("").apply(
    lambda u: f"<img src='{u}' width='120'>" if u else ""
)

cols = ["thumb", "Car_Company_Names", "Car_Models", "Price_USD","ai_image_description"]
display(HTML(df_show[cols].to_html(escape=False, index=False)))


thumb,Car_Company_Names,Car_Models,Price_USD,ai_image_description
,Mazda,CX-9,"$38,000 - $45,000","This is a red Mazda CX-9 SUV with four doors, one of which is open."
,Mitsubishi,Outlander,"$28,000 - $35,000",This is a red Mitsubishi Outlander SUV with four doors on display at an auto show.
,Mazda,CX-5,"$26,000 - $32,000",A red Mazda CX-5 SUV is parked on a road with a mountain in the background.
,Mitsubishi,Eclipse Cross,"$25,000 - $28,000","A red Mitsubishi Eclipse Cross SUV is parked on a cracked, dry lakebed under a clear sky."
,Mazda,CX-30,"$23,000 - $28,000","The photo shows the open trunk of a red Mazda CX-30, revealing the cargo space and rear seats."
,Mazda,CX-3,"$21,000 - $25,000",The image shows a red Mazda CX-3 subcompact crossover SUV.


**RUN A MULTIMODAL QUERY ON AN ARRAY OF OBJECTREFS**

**1) Brand-level themes from car images**

In [ ]:
%%bigquery df_themes --project ecstatic-gantry-470611-m5
WITH grouped AS (
  SELECT
    Car_Company_Names AS company_name,
    ARRAY_AGG(ref IGNORE NULLS LIMIT 20) AS refs,
    ARRAY_AGG(
      REGEXP_REPLACE(uri, r'^gs://([^/]+)/(.+)$',
                     'https://storage.googleapis.com/\\1/\\2')
      IGNORE NULLS LIMIT 4
    ) AS image_urls
  FROM `ecstatic-gantry-470611-m5.cars.cars_multimodal_enriched`
  GROUP BY company_name
),
scored AS (
  SELECT
    company_name,
    AI.GENERATE(
      ('List 2–5 concise themes that characterize this brand’s cars in these images. '
       || 'Return short noun phrases only.' , refs),
      connection_id => 'us.test_connection',
      endpoint => 'gemini-2.5-flash',
      output_schema => 'themes ARRAY<STRING>'
    ).themes AS themes,
    image_urls
  FROM grouped
)
SELECT
  company_name,
  ARRAY_TO_STRING(themes, ', ') AS themes_list,      -- CSV string
  image_urls[OFFSET(0)]          AS image_url        -- single URL
FROM scored
WHERE ARRAY_LENGTH(image_urls) > 0                    -- ensure we have an image
ORDER BY company_name




Query is running:   0%|          |

Downloading:   0%|          |

**Previw: Brand-level themes from car images**

In [ ]:
from IPython.display import HTML, display

dfv = df_themes.copy()
dfv["thumb"] = dfv["image_url"].fillna("").apply(
    lambda u: f"<img src='{u}' width='120'>" if u else ""
)

display(HTML(dfv[["thumb","company_name","themes_list"]].to_html(escape=False, index=False)))




thumb,company_name,themes_list
,ASTON MARTIN,"Luxury vehicles, High performance, Elegant design, Exotic appeal"
,AUDI,"Luxury vehicles, Sleek modern design, Diverse body styles"
,Cadillac,"Luxury design, Modern styling, Bold presence"
,Chevrolet,"Broad vehicle range, American design language, Practical family vehicles, Performance offerings"
,FERRARI,"Sporty convertible, Driving enjoyment, Compact design"
,Ford,"Diverse model range, Practical utility, Modern aesthetics, Performance options"
,GMC,"Utility vehicles, Rugged design, American truck heritage"
,HONDA,"Diverse vehicle range, Contemporary design, Practicality and utility, Family-friendly options"
,HYUNDAI,"Contemporary Styling, Diverse Product Line, Spacious Interiors, Distinctive Design Elements"
,Jeep,"Off-road capability, SUV utility, Rugged design, Adventure readiness"


**2) Multi car image brand summaries and keywords.**

In [ ]:
%%bigquery df_brand_summary --project ecstatic-gantry-470611-m5
WITH grouped AS (
  SELECT
    Car_Company_Names AS company_name,
    -- send up to 16 images per brand to the model
    ARRAY_AGG(ref IGNORE NULLS LIMIT 16) AS refs,
    -- pick a deterministic sample image: most expensive per brand
    ARRAY_AGG(
      REGEXP_REPLACE(uri, r'^gs://([^/]+)/(.+)$',
                     'https://storage.googleapis.com/\\1/\\2')
      IGNORE NULLS
      ORDER BY SAFE_CAST(REGEXP_REPLACE(Price_USD, r'[^0-9.]', '') AS FLOAT64) DESC
      LIMIT 1
    )[OFFSET(0)] AS image_url
  FROM `ecstatic-gantry-470611-m5.cars.cars_multimodal_enriched`
  GROUP BY company_name
),
scored AS (
  SELECT
    company_name,
    image_url,
    -- RUN A MULTIMODAL QUERY ON AN ARRAY OF OBJECTREFS
    AI.GENERATE(
      ('From these images, write ONE concise sentence summarizing this brand’s visual design language '
       || 'and list 3–5 short keywords.' , refs),
      connection_id => 'us.test_connection',
      endpoint => 'gemini-2.5-flash',
      output_schema => 'summary STRING, keywords ARRAY<STRING>'
    ) AS gen
  FROM grouped
)
SELECT
  company_name,
  gen.summary  AS summary,
  gen.keywords AS keywords,
  image_url
FROM scored
ORDER BY company_name



Query is running:   0%|          |

Downloading:   0%|          |

**Preview: Multi car image brand summaries and keywords**

In [ ]:
from IPython.display import HTML, display
import pandas as pd

dfv = df_brand_summary.copy()

# thumbnail
dfv["thumb"] = dfv["image_url"].fillna("").apply(
    lambda u: f"<img src='{u}' width='120'>" if u else ""
)

# keywords → CSV (handle list/tuple/numpy/string gracefully)
def to_list(x):
    try:
        import numpy as np
    except Exception:
        np = None
    if isinstance(x, list): return x
    if isinstance(x, tuple): return list(x)
    if np is not None and isinstance(x, np.ndarray): return x.tolist()
    return [] if x is None else [x] if isinstance(x, str) else []

dfv["keywords_csv"] = dfv["keywords"].apply(lambda ks: ", ".join(map(str, to_list(ks))))

cols = ["thumb", "company_name", "summary", "keywords_csv"]
display(HTML(dfv[cols].to_html(escape=False, index=False)))


thumb,company_name,summary,keywords_csv
,ASTON MARTIN,"The visual design language emphasizes sleek, elegant lines and a powerful, luxurious presence, blending classic refinement with modern performance aesthetics.","Luxury, Sleek, Elegant, Powerful"
,AUDI,"The visual design language is characterized by sleek, modern lines, a premium feel, and sophisticated, minimalist aesthetics.","Sleek, Modern, Premium, Elegant"
,Cadillac,"Cadillac's visual design language is characterized by sharp, angular lines, prominent grilles, and signature vertical lighting elements, conveying a bold and sophisticated presence.","Sharp, Angular, Vertical lighting, Bold, Sophisticated"
,Chevrolet,"Chevrolet's visual design language is diverse and evolving, marked by bold grilles and distinctive, modern styling across its range of vehicles.","Bold, Diverse, Modern, Distinctive, Evolving"
,FERRARI,"The visual design language is characterized by sleek, dynamic forms with flowing lines and a vibrant aesthetic that evokes motion.","Sleek, Dynamic, Flowing, Sporty"
,Ford,"The brand's visual design language is characterized by a blend of modern athleticism and utilitarian robustness across its diverse vehicle lineup, often featuring bold grilles and clean, functional lines.","Modern, Robust, Versatile, Purposeful"
,GMC,"GMC's visual design language is characterized by robust, utilitarian forms with bold, straightforward lines, emphasizing practicality and a powerful presence.","Robust, Utilitarian, Sturdy, Functional, Bold"
,HONDA,"Honda's visual design language balances modern, dynamic lines for passenger cars with robust, practical aesthetics for SUVs and minivans.","Modern, Dynamic, Practical, Robust"
,HYUNDAI,"Hyundai's visual design language demonstrates an evolution from flowing and modern lines to increasingly bold and distinctive forms, characterized by intricate lighting details and practical yet evolving interiors across its diverse vehicle range.","Modern, Diverse, Evolving, Distinctive"
,Jeep,"The brand's visual design language is characterized by robust and versatile forms, emphasizing functionality and adventurous capability across a range of SUVs and trucks.","Robust, Versatile, Off-road, Functional, Adventurous"
